# Artificial Neural Network

### Importing the libraries

In [72]:
# 09/01/2023
# IDE: VSC
# Section 36: Artificial Neural Networks
# Video [335, 339]

# pip install tensorflow==2.14.0

import numpy as np
import pandas as pd
import tensorflow as tf

In [73]:
tf.__version__

'2.14.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [74]:
dataset = pd.read_csv('Churn_Modelling.csv')

# we can exclude columns: RowNumber (0), CustomerId (1), Surname (2)
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [75]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [76]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [77]:
# there is no missing data in the dataset: encoding categorical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Transform male and female into 1 and 0
# X[:, 2] --> all the rows but only the column of index 2
X[:, 2] = le.fit_transform(X[:, 2])

In [78]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [79]:
# Origin: data_preprocessing_tools

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# transformers=[('encoder', ... )] --> encoding transformation
# transformers=[ ... , OneHotEncoder() --> class that will do the transformation
# transformers=[ ..., [1]) ] --> column where you want to apply the encoding
# remainder='passthrough' --> if you do not include this parameter, you don't get the !country columns
# countries: France, Spain, Germany
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')

# it is absolutely compulsory to have X as a numpy array, this will be expected by the future ML models
X = np.array(ct.fit_transform(X))

In [80]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [81]:
# Origin: data_preprocessing_tools

from sklearn.model_selection import train_test_split

# test_size: recommended size of the split
# --> 80% observation on the training set (8 customers go on the training set)
# --> 20% in the test set (2 customers go in the test set)
# random_state: fixing the seed so that we get the same split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [82]:
print(X_train)

[[0.0 1.0 0.0 ... 0 1 124749.08]
 [1.0 0.0 0.0 ... 0 0 41104.82]
 [0.0 1.0 0.0 ... 1 1 45750.21]
 ...
 [1.0 0.0 0.0 ... 1 1 92027.69]
 [1.0 0.0 0.0 ... 1 1 101168.9]
 [0.0 1.0 0.0 ... 1 0 33462.94]]


In [83]:
print(X_test)

[[1.0 0.0 0.0 ... 1 1 97057.28]
 [1.0 0.0 0.0 ... 1 0 66526.01]
 [1.0 0.0 0.0 ... 0 1 90537.47]
 ...
 [0.0 0.0 1.0 ... 0 1 161571.79]
 [0.0 1.0 0.0 ... 1 1 165257.31]
 [0.0 1.0 0.0 ... 1 1 49025.79]]


In [84]:
print(y_train)

[0 0 1 ... 1 0 1]


In [85]:
print(y_test)

[0 0 0 ... 0 0 0]


### Feature Scaling

In [86]:
# Origin: data_preprocessing_tools
# Feature Scaling is absolutely compulsory for DL (Deep Learning)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# fit_transform for all the variables that are not dummy variables in X_train
# fit: will get the mean and standard deviation of each feature
# transform: applies the standardization formula so that all values can be in the same scale
X_train = sc.fit_transform(X_train)

# The features of the test set need to be scaled by the same scaler that was used on the training set
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [87]:
# ANN (Artificial Neural Network)
# neurons here: [CreditScore, Geography, Gender, ... , EstimatedSalary]
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [88]:
# units=6 --> 6 neurons (Hyperparameter)
# activation='relu' --> re_ctifier l_inear u_nit --> rectifier activation function
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [89]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [90]:
# units=1 --> only 1 output neuron is needed to encode a binary outcome
# activation='sigmoid' --> it allows to get a probability prediction
# activation='softmax' --> predicting more than a binary outcome

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [91]:
# the best optimizers are the ones that can perform gradient descent
# stochastic gradient descent: it updates the weights to reduce the loss error between predictions and results

# non-binary loss function--> loss= 'categorical_crossentropy'
ann.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

### Training the ANN on the Training set

In [92]:
# batch_size --> we do the comparison of trainning vs test in batches instead of 1vs1
ann.fit(X_train, y_train, batch_size=32, epochs=50)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5287 - accuracy: 0.7857
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4536 - accuracy: 0.8074
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4319 - accuracy: 0.8169
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4214 - accuracy: 0.8211
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4140 - accuracy: 0.8241
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4068 - accuracy: 0.8289
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3998 - accuracy: 0.8304
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3924 - accuracy: 0.8339
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3851 - accuracy: 0.8355
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3793 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

### Making the Confusion Matrix